# Student Loan Risk with Deep Learning

In [20]:
# Imports
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import load_model
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from pathlib import Path

---

## Prepare the data to be used on a neural network model

### Step 1: Read the `student-loans.csv` file into a Pandas DataFrame. Review the DataFrame, looking for columns that could eventually define your features and target variables.   

In [2]:
# Read the csv into a Pandas DataFrame
file_path = "https://static.bc-edx.com/ai/ail-v-1-0/m18/lms/datasets/student-loans.csv"
loans_df = pd.read_csv(file_path)

# Review the DataFrame
loans_df.head()

,payment_history,location_parameter,stem_degree_score,gpa_ranking,alumni_success,study_major_code,time_to_completion,finance_workshop_score,cohort_ranking,total_loan_score,financial_aid_score,credit_ranking
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,0
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,0
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,0
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,1
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,0


In [3]:
# Review the data types associated with the columns
loans_df.dtypes

payment_history           float64
location_parameter        float64
stem_degree_score         float64
gpa_ranking               float64
alumni_success            float64
study_major_code          float64
time_to_completion        float64
finance_workshop_score    float64
cohort_ranking            float64
total_loan_score          float64
financial_aid_score       float64
credit_ranking              int64
dtype: object

In [4]:
# Check the credit_ranking value counts
loans_df["credit_ranking"].value_counts()

1    855
0    744
Name: credit_ranking, dtype: int64

### Step 2: Using the preprocessed data, create the features (`X`) and target (`y`) datasets. The target dataset should be defined by the preprocessed DataFrame column “credit_ranking”. The remaining columns should define the features dataset.

In [7]:
# Define the target set y using the credit_ranking column
y = loans_df['credit_ranking']

# Display a sample of y
print(y.sample(5).values)



[1 1 1 0 1]


In [8]:
# Define features set X by selecting all columns but credit_ranking
X = loans_df.drop('credit_ranking', axis=1)

# Review the features DataFrame
X.head()



,payment_history,location_parameter,stem_degree_score,gpa_ranking,alumni_success,study_major_code,time_to_completion,finance_workshop_score,cohort_ranking,total_loan_score,financial_aid_score
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4


### Step 3: Split the features and target sets into training and testing datasets.


In [9]:
# Split the preprocessed data into a training and testing dataset
# Assign the function a random_state equal to 1
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)



### Step 4: Use scikit-learn's `StandardScaler` to scale the features data.

In [10]:
# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the scaler to the features training dataset
scaler.fit(X_train)

# Transform training and testing data
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)


---

## Compile and Evaluate a Model Using a Neural Network

### Step 1: Create a deep neural network by assigning the number of input features, the number of layers, and the number of neurons on each layer using Tensorflow’s Keras.

> **Hint** You can start with a two-layer deep neural network model that uses the `relu` activation function for both layers.


In [12]:
# Define the the number of inputs (features) to the model
n_features = X_train_scaled.shape[1]

# Review the number of features
print(f'Number of features: {n_features}')



Number of features: 11


In [13]:
# Define the number of hidden nodes for the first hidden layer
n_nodes_hidden1 = 2 * n_features

# Define the number of hidden nodes for the second hidden layer
n_nodes_hidden2 = int(1.5 * n_features)

# Define the number of neurons in the output layer
n_neurons_output = 1

In [14]:
# Create the Sequential model instance

model = Sequential()
# Add the first hidden layer
model.add(Dense(n_nodes_hidden1, input_dim=n_features, activation='relu'))

# Add the second hidden layer
model.add(Dense(n_nodes_hidden2, activation='relu'))

# Add the output layer to the model specifying the number of output neurons and activation function
model.add(Dense(n_neurons_output, activation='sigmoid'))

In [15]:
# Display the Sequential model summary
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 22)                264       
                                                                 
 dense_1 (Dense)             (None, 16)                368       
                                                                 
 dense_2 (Dense)             (None, 1)                 17        
                                                                 
Total params: 649 (2.54 KB)
Trainable params: 649 (2.54 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


### Step 2: Compile and fit the model using the `binary_crossentropy` loss function, the `adam` optimizer, and the `accuracy` evaluation metric.


In [16]:
# Compile the Sequential model
model.compile(loss='binary_crossentropy', optimizer='adam',  metrics=['accuracy'])

In [17]:
# Fit the model using 50 epochs and the training data
history = model.fit(X_train_scaled, y_train, epochs=50, validation_data=(X_test_scaled, y_test))


Epoch 1/50
40/40 [==============================] - 1s 8ms/step - loss: 0.6622 - accuracy: 0.6294 - val_loss: 0.6263 - val_accuracy: 0.6906
Epoch 2/50
40/40 [==============================] - 0s 3ms/step - loss: 0.6019 - accuracy: 0.7170 - val_loss: 0.5719 - val_accuracy: 0.7344
Epoch 3/50
40/40 [==============================] - 0s 3ms/step - loss: 0.5602 - accuracy: 0.7256 - val_loss: 0.5394 - val_accuracy: 0.7594
Epoch 4/50
40/40 [==============================] - 0s 3ms/step - loss: 0.5391 - accuracy: 0.7420 - val_loss: 0.5242 - val_accuracy: 0.7563
Epoch 5/50
40/40 [==============================] - 0s 3ms/step - loss: 0.5271 - accuracy: 0.7404 - val_loss: 0.5177 - val_accuracy: 0.7531
Epoch 6/50
40/40 [==============================] - 0s 3ms/step - loss: 0.5207 - accuracy: 0.7443 - val_loss: 0.5159 - val_accuracy: 0.7531
Epoch 7/50
40/40 [==============================] - 0s 3ms/step - loss: 0.5158 - accuracy: 0.7482 - val_loss: 0.5122 - val_accuracy: 0.7531
Epoch 8/50
40/40 [==

### Step 3: Evaluate the model using the test data to determine the model’s loss and accuracy.


In [18]:
# Evaluate the model loss and accuracy metrics using the evaluate method and the test data
loss, accuracy = model.evaluate(X_test_scaled, y_test, verbose=2)

# Display the model loss and accuracy results
print(f"Loss: {loss}, Accuracy: {accuracy}")

10/10 - 0s - loss: 0.5170 - accuracy: 0.7594 - 37ms/epoch - 4ms/step
Loss: 0.5169810056686401, Accuracy: 0.7593749761581421


### Step 4: Save and export your model to a keras file, and name the file `student_loans.keras`.


In [19]:
# Set the model's file path
model_file_path = 'student_loans.keras'

# Export your model to a keras file
model.save(model_file_path)




---
## Predict Loan Repayment Success by Using your Neural Network Model

### Step 1: Reload your saved model.

In [21]:
# Set the model's file path
model_file_path = 'student_loans.keras'

# Load the model to a new object
loaded_model = load_model(model_file_path)

### Step 2: Make predictions on the testing data and save the predictions to a DataFrame.

In [22]:
# Make predictions with the test data
predictions = loaded_model.predict(X_test_scaled, verbose=2)

# Display a sample of the predictions
print(predictions[:5])



10/10 - 1s - 551ms/epoch - 55ms/step
[[0.22904143]
 [0.41460034]
 [0.92597455]
 [0.74085706]
 [0.9738236 ]]


In [25]:
# Save the predictions to a DataFrame and round the predictions to binary results

predictions_df = pd.DataFrame(predictions, columns=['Probability'])
predictions_df['predictions'] = predictions_df['Probability'].apply(lambda x: 1 if x >= 0.5 else 0)

print(predictions_df)

     Probability  predictions
0       0.229041            0
1       0.414600            0
2       0.925975            1
3       0.740857            1
4       0.973824            1
..           ...          ...
315     0.812292            1
316     0.148657            0
317     0.246769            0
318     0.947749            1
319     0.948134            1

[320 rows x 2 columns]


### Step 4: Display a classification report with the y test data and predictions

In [26]:
# Print the classification report with the y test data and predictions

binary_predictions_array = predictions_df['predictions'].to_numpy()
print(classification_report(y_test, binary_predictions_array))


              precision    recall  f1-score   support

           0       0.75      0.75      0.75       154
           1       0.77      0.77      0.77       166

    accuracy                           0.76       320
   macro avg       0.76      0.76      0.76       320
weighted avg       0.76      0.76      0.76       320



---
## Discuss creating a recommendation system for student loans

Briefly answer the following questions in the space provided:

1. Describe the data that you would need to collect to build a recommendation system to recommend student loan options for students. Explain why this data would be relevant and appropriate.

2. Based on the data you chose to use in this recommendation system, would your model be using collaborative filtering, content-based filtering, or context-based filtering? Justify why the data you selected would be suitable for your choice of filtering method.

3. Describe two real-world challenges that you would take into consideration while building a recommendation system for student loans. Explain why these challenges would be of concern for a student loan recommendation system.

**1.** 
***Student Financial Profile***: Details like income, employment, credit score, and debts, crucial for evaluating repayment capabilities and loan eligibility.

***Academic Information***: Students' study areas, performance, and institution types, which are vital for assessing loan risks and opportunities.

***Loan Repayment Histories***: Historical data on loan repayments related to students' fields and income levels, aiding in pattern recognition and repayment predictions.

***Market Data***: Information on available loan options, rates, and terms, enabling tailored recommendations.

This data is vital for tailoring loan recommendations to students' unique financial and academic contexts, ensuring relevance and efficacy.


**2.**
***Context-based Filtering***
Given the data specifics, this approach is ideal for customizing loan suggestions. It uses personal and academic details to offer unique, circumstance-fitted loan advice. Unlike collaborative or content-based methods, which depend on user similarities or past preferences, context-based filtering integrates diverse student life facets to craft individualized recommendations.


**3.**
***Data Privacy and Security***
Handling sensitive data necessitates stringent privacy safeguards and compliance with regulations to secure student information.

Dynamic Market Conditions
The fluctuating student loan market requires the system to be agile, adapting to changing rates and new offerings to maintain its relevance and utility.

Addressing these challenges is crucial for the system's success, impacting its effectiveness, ethical integrity, and legal compliance, thereby establishing a reliable and esteemed student loan recommendation platform.